Best-sellers:
https://www.amazon.com/Best-Sellers-Kindle-Store-eBooks/zgbs/digital-text/154606011/ref=zg_bs_unv_kstore_2_3511261011_1

Notes:

Think about where we want exceptions
Make sure to note how we are futzing with data and what we're doing with the exception handling

The problem right now is that with the best-selling, they include some pre-orders, which don't have reviews

We could skip these or we could do a different scrape with this query:

https://www.amazon.com/s?i=digital-text&rh=n%3A133140011%2Cn%3A133141011%2Cn%3A154606011%2Cn%3A3511261011&s=featured-rank&lo=list&qid=1556572094&ref=sr_pg_1

That's ya and teen sorted by "featured"

# NOTE - AZ only lists a single author

Gotta reiterate how to think about query errors

Think about how to handle review text

We can also include a link to the book image

We can also get a bunch of numbers from GR and AZ and compare them?

Note - it will truncate the title if it's long

The GR part doesn't work for all titles; Not sure where it fails

Note: the az list is only for paid ebooks

note: it would be technically better to do the top 100, but it's a PITA

There's something wrong with the indices we are using for testing. I think it should fail b/c I'm passing x-1, but it's not. Check the range thing

Figure out what the F is going on with https://www.amazon.com/Harley-Merlin-Stolen-Magicals-ebook/dp/B07HLKV538/ref=zg_bs_3511261011_40?_encoding=UTF8&psc=1&refRID=M0EFSTR4YYGK220B34QW

so far, it doesn't work for 40-45
works for 46

50 doesn't work --> fix this

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from splinter import Browser
from bs4 import BeautifulSoup as bs
from datetime import datetime
import random
from api_keys import key
import requests
import xml.etree.ElementTree as ET
import urllib.parse as UP

In [2]:
def init_browser():
    """Initializes a splinter Browser object"""
    
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=True)

## Scraping Amazon

In [9]:
def book_scraper(url):
    
    """Scrapes given Amazon url for the book review information.
    Returns a dictionary with full title, whether reviews are available, and review information if so"""
    
    # Open Amazon book url using Splinter
    with init_browser() as browser:
        browser.visit(url)

        # Create Beautiful soup object
        soup = bs(browser.html, "html.parser")
        
        # Get full title
        title = soup.find("span", id="ebooksProductTitle").text.replace("\n","").replace("  ", "")
        
        # Determine if Review information is available. 
        # If not, set rev_available to false and enter None for review information
        # If yes, set rev_available to true and enter review information
        rev_count_string = soup.find("span", id="acrCustomerReviewText")
        asin = soup.find()

        if (rev_count_string == None):
            rev_available = False
            rev_count = None
            rev_score = None
            rev_top = None
        else:
            rev_available = True
            rev_count = int(rev_count_string.text.split(" ")[0].replace(",", ""))
            rev_score_string = soup.find("span", class_="arp-rating-out-of-text a-color-base").text
            rev_score = float(rev_score_string.split(" ")[0])
            rev_top = soup.find("div", class_="a-expander-content reviewText review-text-content a-expander-partial-collapse-content").text

        
        book_dict = {"Title": title, "Amazon Reviews Available": rev_available, "Total Amazon Reviews": rev_count, "Average Amazon Rating": rev_score, "Top Review": rev_top}
        return(book_dict)

In [10]:
def az_scraper(x):

    """Scrapes Amazon.com website for the book rated x on the Amazon Top 50 
    best selling paid young adult and teen books list.
    Returns a dictionary with Title, Author, Amazon book url, total number or reviews, a review score, 
    and the text of the top review"""
    
    # Open Amazon using Splinter
    with init_browser() as browser:
        
        url = 'https://www.amazon.com/Best-Sellers-Kindle-Store-eBooks/zgbs/digital-text/154606011/ref=zg_bs_unv_kstore_2_3511261011_1'
        browser.visit(url)

        # Create Beautiful soup object
        soup = bs(browser.html, "html.parser")
        
        # Create Beautiful soup object for specified book
        book_soup = soup.find_all("span", class_="aok-inline-block zg-item")[x]
        
        # Get Best Selling Book author
        
        try:
            book_author = book_soup.find("a", class_="a-size-small a-link-child").text
        except AttributeError:
            book_author = book_soup.find("div", class_="a-row a-size-small").find("span").text
        
        # Get Best Selling Book link
        book_href = book_soup.find("a")["href"]
        book_url = f"https://www.amazon.com{book_href}"
        
        # Get URL for book image
        book_img = book_soup.find("img")["src"]
        
        # Return dictionary of book information
        book_dict = {**book_scraper(book_url), **{"Author": book_author, "Url": book_url, "Book Image": book_img}}

    return(book_dict)

## Scraping Goodreads

In [4]:
def gr_query(title):

    """Queries Goodreads API for a given book title.
    Returns a dictionary with whether GoodReads review information is available,
    number of GoodReads ratings, and average Goodreads rating"""
    
    # Set base API url
    url = "https://www.goodreads.com/book/title.xml?"

    # Encode title for url query
    title_query = UP.quote_plus(title)
    
    # Compose query url
    query_url = f"https://www.goodreads.com/book/title.xml?&key={key}&title={title_query}"
    
    # Query GoodReads API
    response = requests.get(query_url)
    
    if (response.status_code != requests.codes.ok):
        return({"GoodReads Available": False, "Number of GoodReads Ratings": None, "Average GoodReads Rating": None})
    else:
        # Parse XML from response object
        root = ET.fromstring(response.content)
        for count in root.findall("./book/work/ratings_count"):
            ratings_count = int(count.text)
        for average in root.findall("./book/average_rating"):
            rating_average = float(average.text)

        # Return dictionary of GoodReads rating information
        gr_dict = {"GoodReads Available": True, "Number of GoodReads Ratings": ratings_count, "Average GoodReads Rating": rating_average}
        return(gr_dict)

## Combining Functions

In [11]:
def top_book(x):

    """Scrapes Amazon.com website and queries Goodreads API for the 
    book rated x on the Amazon Top 50 best selling paid young adult and teen books list.
    Returns a dictionary Amazon and GoodReads information"""
    
    # Scrape Amazon for book information
    az_dict = az_scraper(x-1)
    
    # Extract title from dictionary for use in GoodReads query
    title = az_dict["Title"]
    
    # Query GoodReads API for book information
    gr_dict = gr_query(title)
    
    # Return dictionary of book information
    book_dict = {**az_dict, **gr_dict}
    return(book_dict)
    

## Testing

In [ ]:
#This produces a dictionary of book information for the a random book in the top 50 best selling YA books on amazon
# x = random.randint(1,51)
# book_dict = ya_book(x)
# print(book_dict)

In [12]:
top_book(1)

{'Title': 'Cold Waters (Normal, Alabama Book 1)',
 'Amazon Reviews Available': True,
 'Total Amazon Reviews': 485,
 'Average Amazon Rating': 4.2,
 'Top Review': '‘Cold Waters’ was my Amazon First Read choice for April as I love psychological thrillers and I was not disappointed as this was definitely a page turner. Author Debbie Herbert is new to me, however has created a thrilling story of murder and madness set around the dark and cold waters of Alabama.After a summer-night swim 14 year old Violet is found confused, wandering in the forest—and her best friend Ainsley has disappeared. Everyone is accusing Violet of murder however without a body, murder charges won’t stick, yet Violet is sent away to an institution.The story picks up when she returns 10 years later and goes back and forth between that tragic night and the present day - sort of flashback like but mainly her trying to piece things together as she has no solid memory of what happened that night yet it has wreaked havoc on

In [ ]:
#Creates a list of the top 10 books
top_10 = []

for x in range(10):
    top_10.append(ya_book(x))

In [ ]:
next_10 = []

for x in range(10, 20):
    next_10.append(ya_book(x))

In [ ]:
again_10 = []

for x in range(20, 30):
    again_10.append(ya_book(x))

In [ ]:
pu_10 = []

for x in range(30,40):
    pu_10.append(ya_book(x))

In [ ]:
print(top_10)

In [11]:
# This is what needs to be tested
f_10 = []

for x in range(41,51):
    f_10.append(ya_book(x))

AttributeError: 'NoneType' object has no attribute 'text'

In [43]:
x = ya_book(0)

In [44]:
print(x)

{'Title': "The Governess (Ladies of Miss Bell's Finishing School Book 1)", 'Amazon Reviews Available': True, 'Total Amazon Reviews': 23, 'Average Amazon Rating': 4.6, 'Top Review': "The characters are very, very briefly mentioned in On My Honor A Brethren in Arms series. It was a delightful read as we met the four young women but this story focused on Adelaide. Still worried about Adelaide's brother by end of the story but I'm sure his storyline will be in the other series.  Can't wait for book 2.\n", 'Author': 'Elizabeth Johns', 'Url': 'https://www.amazon.com/Governess-Ladies-Bells-Finishing-School-ebook/dp/B07NSPMBMH/ref=zg_bs_3511261011_50/145-3567204-9682019?_encoding=UTF8&psc=1&refRID=SF1MVCBDYW1EKD1N0W9V', 'Book Image': 'https://images-na.ssl-images-amazon.com/images/I/81cWaavKnUL._UX300_PJku-sticker-v7,TopRight,0,-50_OU01__BG0,0,0,0_FMpng_AC_UL200_SR200,200_.jpg', 'GoodReads Available': False, 'Number of GoodReads Ratings': None, 'Average GoodReads Rating': None}


In [24]:
print(x)

{'Title': 'The Paper Magician (The Paper Magician Series, Book 1)', 'Amazon Reviews Available': True, 'Total Amazon Reviews': 4489, 'Average Amazon Rating': 4.0, 'Top Review': 'Not every recommendation is a smash hit.I don\'t feel bad about reading "The Paper Magician." It\'s not a bad book. It\'s not a great book. Mostly what I really feel after finishing "The Paper Magician" is disappointment. Not a lot mind you, but enough that I wish Holmberg had done a slightly better job.Holmberg does have a good introduction. It did not take more than a couple words for me to be whisked away into this new world seeking out adventure and excitement. The first several chapters kept my anticipation levels high as everything started clicking together and I could start to see the outline of something great. Next thing I know, it happened - the great catalyst that sends a story screeching out to the wild blue where heroes are made, and villains are ended. "This is it!" I thought as Ceony is thrust int

In [ ]:
url = "https://www.amazon.com/Harley-Merlin-Stolen-Magicals-ebook/dp/B07HLKV538/ref=zg_bs_3511261011_40?_encoding=UTF8&psc=1&refRID=M0EFSTR4YYGK220B34QW"

In [ ]:
with init_browser() as browser:
    browser.visit(url)

    # Create Beautiful soup object
    soup = bs(browser.html, "html.parser")
    x = soup.find("div", class_="a-section a-spacing-none")
    print(x)

In [ ]:
# Get full title
title = soup.find("span", id="ebooksProductTitle").text.replace("\n","").replace("  ", "")

In [ ]:
x = soup.find("div", class_="a-section a-spacing-none")

In [35]:
x = az_scraper(0)

In [36]:
print(x)

{'Title': 'One Word Kill (Impossible Times Book 1)', 'Amazon Reviews Available': True, 'Total Amazon Reviews': 191, 'Average Amazon Rating': 4.3, 'Top Review': "I almost skipped this book because of the title, but the synopsis didn't match it so I gambled on it. I was 80% right. Nick, our 15 year old protagonist, has just found out he has leukemia and immediately starts chemo. He is a math genius and less comfortable socially. He does have a crew, though, his Dungeons and Dragons game group that meets weekly. They come from both sides of the tracks and Nick is more in the middle. A girl has recently joined the mix. We are given stream of consciousness insight into Nick’s mind. This is very well done. You feel the adolescent determination, scattered thoughts, and tendency to jump into situations without thinking through possible consequences. D&D plays heavily into the story and things in the game start to mysteriously mirror real life. There is a time travel element that drives the plo